In [7]:
#Packages 
import numpy as np
import xarray as xr
import glob
import cmocean.cm as cmo
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import warnings
warnings.filterwarnings("ignore") #turns off annoying warnings
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import FuncFormatter
import mosaic
from matplotlib.colors import SymLogNorm
from matplotlib.colors import LinearSegmentedColormap
from joblib import Parallel, delayed

In [14]:
def interp_mphy(ds):
    # Step 1: Interpolate (average)
    mphys_interp = 0.5 * (
        dsd.chiPhyVerSalt.isel(nVertLevelsP1=slice(0, -1)) +
        dsd.chiPhyVerSalt.isel(nVertLevelsP1=slice(1, None))
    )
    # Step 2: Rename the dimension
    mphys_interp = mphys_interp.rename({'nVertLevelsP1': 'nVertLevels'})
    # Step 3: Assign new coordinates
    mphy_salt = mphys_interp.assign_coords(nVertLevels=dsd.nVertLevels)
    dsd['chiPhyVerSalt'] = mphy_salt

    # Step 1: Interpolate (average)
    mphyt_interp = 0.5 * (
        dsd.chiPhyVerTemp.isel(nVertLevelsP1=slice(0, -1)) +
        dsd.chiPhyVerTemp.isel(nVertLevelsP1=slice(1, None))
    )
    # Step 2: Rename the dimension
    mphyt_interp = mphyt_interp.rename({'nVertLevelsP1': 'nVertLevels'})
    # Step 3: Assign new coordinates
    mphy_temp = mphyt_interp.assign_coords(nVertLevels=dsd.nVertLevels)
    dsd['chiPhyVerTemp'] = mphy_temp

def center_vertices(dsg, hres):
    # Clean up the indexing so the domain is centered for plotting

    # Original vertices (x,y) arrays
    xv = dsg.xVertex.values
    yv = dsg.yVertex.values
    
    # Compute the new vertices based on your filtering and offset logic:
    
    # 1. Extract verticesOnCell (assuming shape: [nCells, maxVerticesPerCell])
    voc = dsg.verticesOnCell.values
    
    # 2. Build verts as in your snippet (shape: [nCells, maxVerticesPerCell, 2])
    verts = np.dstack((xv[voc - 1], yv[voc - 1]))
    nverts = np.sum(voc != 0, axis=1)
    verts_list = [vert[:n] for vert, n in zip(verts, nverts)]
    
    # 3. Filter verts using np.ptp
    idx = [np.ptp(vert[:, 0]) < 50000 for vert in verts_list]
    verts_filtered = np.array(verts_list)[idx]
    
    # 4. Copy and apply offsets
    nuverts = verts_filtered.copy()
    m_dsx = hres
    nuverts[:, :, 0] += m_dsx / 2
    nuverts[:, :, 1] -= m_dsx
    
    # Now, update dsg's xVertex and yVertex arrays accordingly:
    # But note: verts are grouped by cell, so we need to flatten and assign properly.
    
    # Because verts_filtered is a filtered subset of verts_list,
    # you should update only those cells where idx is True.
    
    # Get indices of cells that passed filter
    filtered_cells = np.where(idx)[0]
    
    # Create copies of xVertex and yVertex to modify
    new_xVertex = xv.copy()
    new_yVertex = yv.copy()
    
    # Loop over filtered cells and update the corresponding vertex coords
    for cell_i, verts_cell in zip(filtered_cells, nuverts):
        nv = verts_cell.shape[0]  # number of vertices for this cell
        vertex_inds = voc[cell_i, :nv] - 1  # zero-based vertex indices for this cell
        new_xVertex[vertex_inds] = verts_cell[:, 0]
        new_yVertex[vertex_inds] = verts_cell[:, 1]
    
    # Assign back to dsg (if dsg is an xarray Dataset or DataArray)
    dsg['xVertex'].values = new_xVertex
    dsg['yVertex'].values = new_yVertex

In [15]:
# Choose one root directory, useful for multiple resolutions
rootdir = '/pscratch/sd/d/dylan617/bichan/mpaso/1km/'

# Load datasets
dso = xr.open_dataset(rootdir + 'output.nc', chunks={'Time': 1}).isel(Time=slice(1, None))
dsg = xr.open_dataset(rootdir + 'channel_1km_init.nc')
dsd = xr.open_dataset(rootdir + 'analysis_members/discreteVarianceDecay.0001-01-01.nc')
interp_mphy(dsd) # Call interpolation of physical mixing
center_vertices(dsg, 1e3) # Call fixing of vertices, second arg is horz res in meters

# Normalize relative vorticity and divergence
dso['normalizedRelativeVorticity'] = dso.relativeVorticityCell / dsg.fCell
dso['normalizedDivergence'] = dso.divergence / dsg.fCell

# In my streams file, I output every 2 hours. 

In [16]:
# Mosaic descriptor object, needed to plot the polygons
descriptor = mosaic.Descriptor(dsg,use_latlon=False)

# Used for plotting mnum on a diverging log colorbar
# Otherwise the extrema look the same for most cmocean maps. That is, 
# you can't tell dark reds or blues apart. 
def truncate_colormap(cmap, minval=0.0, maxval=0.8, n=256):
    """Truncates a colormap to use only part of the range."""
    new_cmap = LinearSegmentedColormap.from_list(
        f'trunc({cmap.name},{minval:.2f},{maxval:.2f})',
        cmap(np.linspace(minval, maxval, n))
    )
    return new_cmap

curl = truncate_colormap(plt.get_cmap('cmo.curl'), 0.1, 0.9)

# Helper to safely compute log10 for plotting, otherwise it is masked for 
# very small values
def safe_log10(data, floor=1e-10):
    """Compute log10 of xarray DataArray safely, keeping dimensions and attrs."""
    # Replace nonpositive or nonfinite values with floor
    data_clean = data.where((data > 0) & np.isfinite(data), other=floor)
    return np.log10(data_clean)

In [18]:
def plot_frame(t):
    z = 0
    fig, axes = plt.subplots(1, 4, figsize=(11, 5), constrained_layout=True, dpi = 200)
    
    # Panel (a): Salinity
    c = mosaic.polypcolor(axes[0], descriptor,
                          dso.salinity.isel(nVertLevels=z, Time=t),
                          cmap=cmo.haline, vmin=15, vmax=50,
                          antialiaseds=False)
    fig.colorbar(c, ax=axes[0], extend='both', label = '[g/kg]')
    
    # Panel (b): Relative Vorticity
    c = mosaic.polypcolor(axes[1], descriptor,
                          dso.relativeVorticityCell.isel(nVertLevels=z, Time=t).values / dsg.fCell,
                          cmap=cmo.balance, vmin=-3, vmax=3,
                          antialiaseds=False)
    fig.colorbar(c, ax=axes[1], extend='both')
    
    # Panel (c): M_num
    data_c = dsd.chiSpurSaltBR08.isel(nVertLevels=z, Time=t)
    data_clean = data_c.where(np.isfinite(data_c), other=0.0)
    absmax = 1e-3
    linthresh = 1e-6
    norm = SymLogNorm(linthresh=linthresh, vmin=-absmax, vmax=absmax)
    
    c = mosaic.polypcolor(axes[2], descriptor, data_clean,
                      cmap=curl, norm=norm,
                      antialiaseds=False)
    fig.colorbar(c, ax=axes[2], extend='both', label = r'[(g/kg)$^2$/s]')

    
    # Panel (d): M_phy
    data_d = safe_log10(dsd.chiPhyVerSalt.isel(nVertLevels=z, Time=t))
    c = mosaic.polypcolor(axes[3], descriptor, data_d,
                          cmap=cmo.matter_r, vmin=-7, vmax=-3,
                          antialiaseds=False)
    fig.colorbar(c, ax=axes[3], extend='both', label = r'[(g/kg)$^2$/s]',
                 format=tick.FormatStrFormatter('$10^{%.1f}$'))
    
    for ax in axes:
        ax.set_xticks(np.arange(0, 350000, 50000))
        ax.set_yticks(np.arange(0, 350000, 50000))
        ax.set_xlim(0, 100000)
        ax.set_ylim(0, 300000)
        ax.set_aspect(1.0)
        ax.set_xticklabels([str(x / 1000.0) for x in ax.get_xticks()])
        ax.set_yticklabels([str(y / 1000.0) for y in ax.get_yticks()])
        ax.set_xlabel('Along-chan. dist. [km]', fontsize=12)
        ax.set_ylabel('Cross-chan. dist. [km]', fontsize=12)
    
    for ax in axes[1:]:
        ax.set_ylabel('')
        ax.set_yticklabels('')
    
    axes[0].set_title('(a) Salinity')
    axes[1].set_title('(b) $(\partial_x v - \partial_y u)/f$')
    axes[2].set_title(r'(c) $\mathcal{M}_{num}$')
    axes[3].set_title(r'(d) $\mathcal{M}_{phy}$')
    
    plt.savefig(
        f'/pscratch/sd/d/dylan617/animations/bichan_1km/surface_fields_{t}.png',
        dpi=400, bbox_inches='tight'
    )

# Run 8 plots in parallel at once (adjust n_jobs based on your CPU). This is magic!
Parallel(n_jobs=8)(delayed(plot_frame)(t) for t in range(0, len(dso.Time)))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,